In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests

# 셀레니움에 다양한 옵션을 적용시키기 위한 패키지
from selenium.webdriver.chrome.options import Options

# 크롬 드라이버 매니저를 실행시키기 위해 설치해주는 패키지
from selenium.webdriver.chrome.service import Service
# 자동으로 크롬 드라이브를 최신으로 유지해주는 패키지 
from webdriver_manager.chrome import ChromeDriverManager
# 클래스, 아이디, css_selector를 이용하고자 할때
from selenium.webdriver.common.by import By
# 키보드 입력
from selenium.webdriver.common.keys import Keys
# 마우스 이동 등의 액션을 위한 부분
from selenium.webdriver.common.action_chains import ActionChains

# 페이지로딩을 비롯한 인터벌을 주기 위해 필수적으로 써야 되는 녀석
import time

# 웹브라우저에서 접속들어온 클라이언트의 단말기가 뭐인지 알려주는 부분
user = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"

options_ = Options()
options_.add_argument(f"User-Agent={user}")

# 크롬 시크릿 모드창으로 열기
options_.add_argument("incognito")
options_.add_experimental_option("detach", True)
options_.add_experimental_option("excludeSwitches", ["enable-logging"])

# 크롬 드라이버 매니저를 자동으로 설치되도록 실행시키는 코드
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options_)

# 접속하려는 주소 입력하여 이동
url = "https://store.steampowered.com/"
driver.get(url)
time.sleep(0.5)

# 언어설정 바꾸기 (파머님께서 도와주신 코드에서 힌트를 얻어 추가함)
lang_change = driver.find_element(By.CSS_SELECTOR, ".pulldown.global_action_link")
lang_change.click()
time.sleep(0.5)

# 4번째가 한글이다. 이걸 클릭하게 하자. (파머님께서 도와주신 코드에서 힌트를 얻어 추가함)
lang_select = driver.find_elements(By.CSS_SELECTOR, ".popup_body.popup_menu > a")[3]
lang_select.click()
time.sleep(3)

# hover할 요소 찾기 (파머님 도움 얻음)
element_to_hover_over = driver.find_elements(By.CSS_SELECTOR,".supernav_container > a")[0]
time.sleep(0.5)

# ActionChains 객체 생성 (파머님 도움 얻음)
actions = ActionChains(driver)
time.sleep(0.5)

# 마우스를 요소 위로 이동시키기 (파머님 도움 얻음)
actions.move_to_element(element_to_hover_over).perform()
time.sleep(0.5)

# 드롭다운 메뉴의 4번째 항목 클릭 (파머님 도움 얻음)
menu_item = driver.find_element(By.LINK_TEXT, "통계") # 파머님께서 도움주신건 영문버전이라 텍스트를 바꿔야 했다
menu_item.click()
time.sleep(2)

# 가장 최근의 주간차트로 이동하기 (CSS셀렉터로 안되어 링크텍스트 클릭으로 해봤는데 성공했다)
chart_item = driver.find_element(By.LINK_TEXT, "2024년 1월 30일") # 해당부분은 스팀에서 주간 차트가 변경될 때 날짜 부분을 수정입력한 뒤 시도해도 작동한다.
chart_item.click()
time.sleep(1.5)

# 스크롤을 내리면 100개 모두 보기가 있다. 클릭하자
button_1 = driver.find_element(By.CLASS_NAME, "DialogButton._DialogLayout.Primary.Focusable")
button_1.click()
time.sleep(2)

# 현재 페이지의 HTML 가져오기 (여기서는 뽑아오려는 페이지의 모든 정보들)
html = driver.page_source

# BeautifulSoup을 사용하여 HTML 파싱
soup = BeautifulSoup(html, 'html.parser')

weeklychart_data = driver.find_elements(By.CSS_SELECTOR , ".weeklytopsellers_TableRow_2-RN6")

result = []
for i in weeklychart_data :
    # 차트리스트 뽑아보기
    ranking = i.find_element(By.CLASS_NAME, 'weeklytopsellers_RankCell_34h48').text
    thumbnail = i.find_element(By.CSS_SELECTOR, 'img.weeklytopsellers_CapsuleArt_2dODJ')
    src = thumbnail.get_attribute('src')
    title = i.find_element(By.CLASS_NAME, 'weeklytopsellers_GameName_1n_4-').text
    if i.find_element(By.CLASS_NAME, 'weeklytopsellers_PriceWidget_30rPO').text:
        price = i.find_element(By.CLASS_NAME, 'salepreviewwidgets_StoreSalePriceBox_Wh0L8').text
    else:
        price = '성인인증 후 확인 가능'
    change_rank = i.find_element(By.CLASS_NAME, 'weeklytopsellers_ChangeCell_1ZdIh').text
    weeks_in_a_row = i.find_element(By.CLASS_NAME, 'weeklytopsellers_WeeksCell_xm7Jp').text

    result.append([ranking, src, title, price, change_rank, weeks_in_a_row])

for i in result:
    print(i)